### Practical Lab: Scanning ports and forging packets

In [ ]:
# Using nmap to determine another machine's OS
# on target: ifconfig gives us the target's IP address
# on machine: -O option to obtain OS, -sV version analysis
nmap -O 192.168.1.2

In [ ]:
# Scanning ports:
# TCP with option -sS
nmap -sS 192.168.1.2

21 tcp ftp  
22 tcp ssh  
53 tcp domain  
80 tcp http  
111 tcp rpcbind  
389 tcp ldap  
2049 tcp nfs

In [ ]:
# UDP scan: option -sU
nmap -sU 192.168.1.2

53 domain  
111 domain  
500 rpcbind  
1701 L2TP

In [ ]:
# Other scan options:
# -sN: null scan: sends TCP packets with no flags enabled
# -sF: fin scan: sends TCP packets with only FIN flag enabled
# -sX: Xmas scan: sends TCP packets with FIN, URG, PSH flags enabled

# all of these scans return:
# RST: closed port
# nothing(silence): open port

#### Creating a script to use iptables

In [ ]:
#!/bin/sh

iptables -F # flushes: to empty the table

# aiming at all packets going through our machine and having 192.168.1.2 as destination and on tcp port
# we create a mask with all bits to 1 and compare it to the mask with the syn bit set to 1
# we drop it if it fits our requirements
iptables -A FORWARD -d 192.168.1.2 -p tcp --tcp-flags ALL SYN -j DROP

# same but this time we compare our mask all with the mask having the FIN, PSH and URG bits set to 1
iptables -A FORWARD -d 192.168.1.2 -p tcp --tcp-flags ALL, FIN,PSH,URG -j DROP

Using scapy to find open ports using TCP SYN scans

In [ ]:
from scapy.all import *

ports = []

for i in range(1, 500):
  p = sr1(IP(dst="192.168.1.2")/TCP(dport=i,flags="S"))
  if p[TCP].flags=="SA": # SYN-ACK flag meaning port is open
    ports += [p[TCP].sport]

print("Open ports", str(ports))

In [ ]:
from scapy.all import *

ports = []

for i in range(1, 500):
  p = sr1(IP(dst="192.168.1.2")/TCP(dport=i,flags="FPU"), timeout=1) # christmas scan
  if not p: # no answer means open port (or filtered)
    ports += [i]

print("Open ports", str(ports))

#### ARPspoof: cache poisonning
Man in the middle attack

In [ ]:
echo 1 > /proc/sys/net/ipv4/ip_forward
sysctl -p

# using ifconfig again to obtain the eth0 network interface and system IP address
ifconfig

# to find host's IP: displays the current ARP table
arp -n

# commands: sending fake ARP replies to the victim: <host_ip>
# our machine has the network interface <interface>
# we are pretending to be <system_ip>
arpspoof -i <interface> -t <host_ip> <system_ip>

# to save and view it:
tcpdump -i <interface> -l > cap.txt

Using arpwatch

In [ ]:
# enabling arpwatch
/etc/init.d/arpwatch start

# watching a certain interface
arpwatch -i eth0

# checking:
# we should see that more than one interface is associated to a certain IP address
# (the previously real one and the new fake one)
cat /var/lib/arpwatch/arp.dat

# to stop arpwatch
/etc/init.d/arpwatch stop

#### A user regurlaly connects to a certain machine (S) from another machine (D)
Find the credentials

In [ ]:
# capturing logs:
tcpdump -w captures.pcap

# opening the file on wireshark
# right click on telnet packet and "following tcp stream"
# obtaining plaintext credentials

Learning about rwwwshell:
Someone tries to connect from S to O every two seconds to start a remote shell. Using wwww-shell to enable the attack, find the the process opening the backdoor on S

Method:
Here is the translation:

- search with `netstat`: to see the connections
- search with `pstree` who started it: we obtain a tree and we can see how a certain program is started
- find the PID
- find the parent PID: ps -edf |grep <PID>: gives a table with parent PID
- find where the parent is launched and by whom
- remove the launch line
- kill the parent process: kill -9 <pid>
- kill the child process
- verify: netstat -taupn

### Keeping logs of iptables actions

In [ ]:
# keeps log of all iptables actions
iptables -A OUTPUT -j LOG
iptables -A INPUT -j LOG
iptables -A FORWARD -j LOG

# keeping a log of only reject packets
iptables -N LOG_DROP
iptables -A LOG_DROP -j LOG --log-prefix '[IPTABLES DROP] : '
iptables -A LOG_DROP -j DROP